In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random

In [ ]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [ ]:
req1 = requests.get("https://www.eventscribe.net/2022/nacfcprogram/agenda.asp?pfp=Browse%20by%20Day")
soup = BeautifulSoup(req1.content, 'html.parser')

In [ ]:
c = 0
for i in soup.find_all(class_=re.compile('list-group-item loadbyurl.*'))[0:]:
    code = i.get('data-presid')
    
    session_url = f'https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?efp=Vk9PSkdNRFExNjM2Ng&PresentationID={code}&rnd=0.7055475&mode=presinfo'
    req = requests.get(session_url)
    soup = BeautifulSoup(req.content, 'html.parser')

    print(session_url)
    session_title = soup.find('h1').text.strip()
    session_date = soup.find(class_='pull-left pres-tidbit tipsytip').text.strip()
    ses_time_loc = soup.find_all(class_='pull-left pres-tidbit')
    session_time = ses_time_loc[0].text.strip()
    
    if len(ses_time_loc) > 1:
        session_loc = ses_time_loc[1].text.strip()
    else:
        print('no location....')
        session_loc = ''
        
    sess_text = ''
    session_author = ''
    affiliation = ''
    sess_type = ''
    category = ''
    cat = soup.find_all(class_='trackname innertracks')
    if len(cat) < 1:
        cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')
    
    try:
        sess_type = soup.find(style='color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8;').text.strip()
    except:
        try:
            sess_type = soup.find(style='margin-top:10px;').text.strip()
        except:
            sess_type = ''

    for i in cat:
        category += i.text.strip() + '; '

    auth = soup.find(class_='speakers-wrap')

    for s_i in auth.find_all(class_='row'): # author affiliation
        aut = s_i.find(class_='col-xs-12 col-md-6').a.text
        aff = s_i.find(class_="text-muted prof-text")
        aff = re.split('<br.*?>', str(aff))
        aff = ' '.join(aff)
        if re.search('<p class="text-muted prof-text">(.*?)</p>', aff):
            aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff).group(1)
        else:
            aff = ' '
        session_author += aut.strip() + '; '
        affiliation += aff + '; '
        
    try: # abs_text with giving spaces
        sess_text = soup.find(class_='PresentationAbstractText mar-top')
        sess_text = str(sess_text)
        sess_text = re.sub('<.*?>', ' ', sess_text)
    except:
        sess_text = ''
    if sess_text == None:
        sess_text = ''

    if session_title == sess_type:
        sess_type = ' '
        print('check if.......')
        
    category = re.sub(sess_type+'; ', '', category)
    print(f'session title :- {session_title}')
    print(f'session type :- {sess_type}')
    print(f'category :- {category}')
    print(f'session date :- {session_date}')
    print(f'session time :- {session_time}')
    print(f'session location :- {session_loc}')
    print(f'session author :- {session_author}')
    print(f'session aff :- {affiliation}')
    print(f'session text :- {sess_text}')

    print('======================='*5)
    
    dic['session_number'].append(session_number)
    dic['name'].append('Session')
    dic['source_id'].append('')
    dic['manual_id'].append('')
    dic['article_title'].append(session_title)
    dic['url'].append(session_url)
    dic['authors'].append(session_author)
    dic['author_affiliation'].append(affiliation)
    dic['abstract_text'].append(sess_text)
    dic['date'].append(session_date)
    dic['start_time'].append(session_time)
    dic['end_time'].append('')
    dic['location'].append(session_loc)
    dic['session_title'].append(session_title)
    dic['session_type'].append(sess_type)
    dic['category'].append(category)
    dic['sub_category'].append('')
    dic['disclosure'].append('')
    
    
    for i in re.split("li class='list-group-item '", req.text)[1:]: # article links

        url = re.search("data-presid='(.*?)' data-url", i).group(1)
        article_url = f'https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?Mode=presInfo&PresentationID={url}'
        print(article_url)
        req_art = requests.get(article_url)
        soup = BeautifulSoup(req_art.content, 'html.parser')
        
        article_cat = category
        cat = soup.find_all(style='background:#489CD8; color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px;')
        if len(cat) < 1:
            cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')
        
        for i in cat:
            article_cat += i.text.strip() + '; '
            
        article_title = soup.find('h1', style='margin-top:10px;').text.strip()
        article_date = soup.find('div', class_='pull-left pres-tidbit tipsytip').text.strip()
        article_author = ''
        article_aff = ''
        article_abs_text = ''
        time_loc = soup.find_all(class_='pull-left pres-tidbit')
        art_time = time_loc[0].text.strip()
        art_auth = soup.find(class_='speakers-wrap')

        for a_i in art_auth.find_all(class_='row'): # author affiliation
            aut = a_i.find(class_='col-xs-12 col-md-6').a.text
            aff = a_i.find(class_="text-muted prof-text")
            aff = re.split('<br.*?>', str(aff))
            aff = ' '.join(aff)
            if re.search('<p class="text-muted prof-text">(.*?)</p>', aff):
                aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff).group(1)
            else:
                aff = ' '
            article_author += aut.strip() + '; '
            article_aff += aff + '; '
        try:
            article_abs_text = soup.find(class_='PresentationAbstractText mar-top')
            article_abs_text = str(article_abs_text)
            article_abs_text = article_abs_text.replace('<b>', '')
            article_abs_text = article_abs_text.replace('</b>', '')
            article_abs_text = re.split('<br.*?>', article_abs_text)
            article_abs_text = ' '.join(article_abs_text)
            article_abs_text = re.search('<div class="PresentationAbstractText mar-top">(.*?)</div>', article_abs_text).group(1)
        except:
            article_abs_text = ''

        print(article_title)
        print(article_date)
        print(art_time)
        print(article_author)
        print(article_aff)
        print(f'article text :- {article_abs_text}')
        print(f'dusra column :- {article_cat}')
        
        dic['session_number'].append(session_number)
        dic['name'].append('Presentation')
        dic['source_id'].append('')
        dic['manual_id'].append('')
        dic['article_title'].append(article_title)
        dic['url'].append(article_url)
        dic['authors'].append(article_author)
        dic['author_affiliation'].append(article_aff)
        dic['abstract_text'].append(article_abs_text)
        dic['date'].append(article_date)
        dic['start_time'].append(art_time)
        dic['end_time'].append('')
        dic['location'].append(session_loc)
        dic['session_title'].append(session_title)
        dic['session_type'].append(sess_type)
        dic['category'].append(category)
        dic['sub_category'].append('')
        dic['disclosure'].append('')
       
    c += 1
    print(c)

In [ ]:
df = pd.DataFrame(dic)
df

In [ ]:
df.to_excel('nacfc_.xlsx')